In [15]:

import importlib

# ENVIRONMENT
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import dotenv
import os

dotenv.load_dotenv('.env')
MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.path.join(os.getcwd(), os.environ["CATALOG_PATH"])

# nautilus_trader imports

from nautilus_trader.model.identifiers import Venue, InstrumentId, Symbol
from nautilus_trader.model.data import Bar, BarType, QuoteTick
from nautilus_trader.config import BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.engine import BacktestResult
from nautilus_trader.trading.strategy import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos, maybe_unix_nanos_to_dt, unix_nanos_to_dt
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.cache.cache import Cache
from nautilus_trader.model.position import Position
from nautilus_trader.model.objects import Price
from decimal import Decimal

# other imports
from pandas import Timestamp
import importlib
import mplfinance as mpf
import matplotlib.pyplot as plt

# my packages
import put101.indicators as indicators
importlib.reload(indicators)

import strategies
importlib.reload(strategies)

import strategies.bollinger_cluster
importlib.reload(strategies.bollinger_cluster)
from strategies.bollinger_cluster import BollingerCluster


import put101.utils as utils
importlib.reload(utils)


# ---------------- CONFIGURATION ----------------
catalog = ParquetDataCatalog(CATALOG_PATH)
start = dt_to_unix_nanos(pd.Timestamp("2023-11-01 00:00:00"))
end = start + pd.Timedelta(days=90).value 

venue_str = "SIM_EIGHTCAP"
venue = Venue(venue_str)
symbol_str = "EURUSD"
symbol = Symbol(symbol_str)
instrument_id_str = f"EURUSD.{venue}"

instrument_id = InstrumentId(symbol, venue)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
venue_configs = [
    BacktestVenueConfig(
        name=venue_str,
        oms_type="HEDGING",
        account_type="MARGIN",
        base_currency="USD",
        starting_balances=["10_000 USD"],
    ),
]

data_configs = [
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=QuoteTick,
        instrument_id=instrument_id,
        start_time=start,
        end_time=end,
    ),
]

strategies = [
    ImportableStrategyConfig(
        strategy_path="strategies.bollinger_cluster:BollingerCluster",
        config_path="strategies.bollinger_cluster:BollingerClusterConfig",
        config=dict(
            instrument_id=instrument_id.value,
            bar_type=f"{instrument_id}-5-MINUTE-BID-INTERNAL",
            bb_params=[
                (5, 2),
                (60, 2),
            ],
        ),
    ),
]

configs = [BacktestRunConfig(
    engine=BacktestEngineConfig(
        strategies=strategies,
    ),
    data=data_configs,
    venues=venue_configs,
)]

node = BacktestNode(configs)
print(strategies)

[ImportableStrategyConfig(strategy_path='strategies.bollinger_cluster:BollingerCluster', config_path='strategies.bollinger_cluster:BollingerClusterConfig', config={'instrument_id': 'EURUSD.SIM_EIGHTCAP', 'bar_type': 'EURUSD.SIM_EIGHTCAP-5-MINUTE-BID-INTERNAL', 'bb_params': [(5, 2), (60, 2)]})]


In [17]:
results = node.run()

2023-11-07T03:05:00.320000000Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2023-11-07T03:05:00.320000000Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2023-11-07T03:05:00.320000000Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2023-11-07T03:05:00.320000000Z [INFO] BACKTESTER-001.BacktestEngine: Copyright (C) 2015-2024. All rights reserved.
2023-11-07T03:05:00.320000000Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2023-11-07T03:05:00.320000000Z [INFO] BACKTESTER-001.BacktestEngine: 
2023-11-07T03:05:00.320000000Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2023-11-07T03:05:00.320000000Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2023-11-07T03:05:00.320000000Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2023-11-07T03:05:

2023-11-02T22:20:00.612000000Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


_id=O-20231101-0830-001-000-10, tags=TAKE_PROFIT).
2023-11-01T08:30:00.236000000Z [INFO] BACKTESTER-001.BollingerCluster: <--[EVT] OrderPendingCancel(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231101-0830-001-000-12, venue_order_id=None, account_id=SIM_EIGHTCAP-001, ts_event=1698827400000000000).
2023-11-01T08:30:00.236000000Z [WARN] BACKTESTER-001.BollingerCluster: <--[EVT] OrderRejected(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231101-0830-001-000-11, account_id=SIM_EIGHTCAP-001, reason='REJECT OTO from O-20231101-0830-001-000-10', ts_event=1698827400000000000).
2023-11-01T08:30:00.236000000Z [INFO] BACKTESTER-001.BollingerCluster: Handling contingencies for O-20231101-0830-001-000-11.
2023-11-01T08:30:00.236000000Z [INFO] BACKTESTER-001.BollingerCluster: Processing OUO contingent order O-20231101-0830-001-000-12, leaves_qty=Quantity('60345'), contingent_order.leaves_qty=Quantity('60345').
2023-11-01T08:30:00.236000000Z [INFO] BACKTESTER-001.BollingerCluster:

2023-11-06T21:25:00.576000000Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).
2023-11-08T00:20:04.346000000Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


SD, instrument_id=EURUSD.SIM_EIGHTCAP)], event_id=a2f85692-c17e-4ed1-ae8b-3e6f7a463bbe).
2023-11-02T22:20:00.612000000Z [INFO] BACKTESTER-001.BollingerCluster: <--[EVT] OrderAccepted(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231101-1115-001-000-27, venue_order_id=SIM_EIGHTCAP-1-006, account_id=SIM_EIGHTCAP-001, ts_event=1698837300569000000).
2023-11-02T22:20:00.612000000Z [INFO] BACKTESTER-001.Portfolio: EURUSD.SIM_EIGHTCAP margin_init=1_587.55 USD
2023-11-02T22:20:00.612000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=SIM_EIGHTCAP-001, account_type=MARGIN, base_currency=USD, is_reported=False, balances=[AccountBalance(total=10_028.38 USD, locked=3_173.76 USD, free=6_854.62 USD)], margins=[MarginBalance(initial=1_587.55 USD, maintenance=1_586.21 USD, instrument_id=EURUSD.SIM_EIGHTCAP)], event_id=7ec68c9b-667c-4947-8ce9-3f14c4bc220d).
2023-11-02T22:20:00.612000000Z [INFO] BACKTESTER-001.BollingerCluster: <--[EVT] OrderFilled(instrument_id=EURUSD.SIM_

2023-11-08T07:48:20.128000000Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


nt order O-20231101-1725-001-000-51, leaves_qty=Quantity('50233'), contingent_order.leaves_qty=Quantity('50233').
2023-11-08T07:46:49.603000000Z [INFO] BACKTESTER-001.BollingerCluster: Cancelling order LimitOrder(BUY 50_233 EURUSD.SIM_EIGHTCAP LIMIT @ 1.05510 GTC, status=PENDING_CANCEL, client_order_id=O-20231101-1725-001-000-51, venue_order_id=None, position_id=None, contingency_type=OUO, linked_order_ids=[O-20231101-1725-001-000-50], parent_order_id=O-20231101-1725-001-000-49, tags=TAKE_PROFIT).
2023-11-08T07:46:49.603000000Z [WARN] BACKTESTER-001.BollingerCluster: Cannot cancel order: state is PENDING_CANCEL, LimitOrder(BUY 50_233 EURUSD.SIM_EIGHTCAP LIMIT @ 1.05510 GTC, status=PENDING_CANCEL, client_order_id=O-20231101-1725-001-000-51, venue_order_id=None, position_id=None, contingency_type=OUO, linked_order_ids=[O-20231101-1725-001-000-50], parent_order_id=O-20231101-1725-001-000-49, tags=TAKE_PROFIT).
2023-11-08T07:46:49.927000000Z [WARN] BACKTESTER-001.BollingerCluster: <--[EVT]

2023-11-13T17:40:00.465000000Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).
2024-01-25T19:09:51.935895001Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).
2024-01-25T19:09:51.938812001Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).
2024-01-25T19:09:51.938893001Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).
2024-01-25T19:09:51.938953001Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).
2024-01-25T19:09:51.939097001Z [ERROR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from boo

[DEFAULT] GTC, status=SUBMITTED, client_order_id=O-20231101-1825-001-000-59, venue_order_id=None, position_id=None, contingency_type=OUO, linked_order_ids=[O-20231101-1825-001-000-60], parent_order_id=O-20231101-1825-001-000-58, tags=STOP_LOSS).
2023-11-08T07:48:09.669000000Z [INFO] BACKTESTER-001.BollingerCluster: parent_filled_qty=Quantity('50233').
2023-11-08T07:48:13.093000000Z [INFO] BACKTESTER-001.Portfolio: EURUSD.SIM_EIGHTCAP net_position=50233
2023-11-08T07:48:13.093000000Z [INFO] BACKTESTER-001.Portfolio: EURUSD.SIM_EIGHTCAP margin_maint=1_590.14 USD
2023-11-08T07:48:13.093000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=SIM_EIGHTCAP-001, account_type=MARGIN, base_currency=USD, is_reported=False, balances=[AccountBalance(total=10_045.60 USD, locked=1_590.14 USD, free=8_455.46 USD)], margins=[MarginBalance(initial=0.00 USD, maintenance=1_590.14 USD, instrument_id=EURUSD.SIM_EIGHTCAP)], event_id=8e73e285-1210-47ac-af79-80a7c7fe5047).
2023-11-08T07:48:13.

In [ ]:
res = results[0]
backtest_start = maybe_unix_nanos_to_dt(res.backtest_start)
backtest_end = maybe_unix_nanos_to_dt(res.backtest_end)
res

In [ ]:
import put101.vizz as vizz

# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib qt


engine = node.get_engine(res.run_config_id)
strategy: BollingerCluster = engine.trader.strategies()[0]
cache: Cache = strategy.cache

main_t, main_s = strategy.get_main_plottable_indicators()
extra_plots = strategy.get_extra_plots()

layout = utils.get_layout(
                    res=res,
                    bars=strategy.bars,
                    overlay_indicators=main_t,
                    overlay_indicator_styles=main_s,
                    extra_plots=extra_plots,
                    positions=strategy.cache.positions(),
)

vizz.reset_output()
vizz.show(layout)

In [ ]:
engine = node.get_engine(res.run_config_id)
strategy: BollingerCluster = engine.trader.strategies()[0]
cache: Cache = strategy.cache


In [ ]:
pd.set_option("display.max_colwidth", None)
positions = strategy.cache.positions()
df = pd.DataFrame([p.to_dict() for p in positions])

In [ ]:
df

In [ ]:
strategy.portfolio_equity

In [ ]:
from nautilus_trader.model.instruments import Instrument
Instrument.base_to_dict(strategy.instrument)


In [ ]:
positions = strategy.cache.positions()
df = pd.DataFrame([p.to_dict() for p in positions])
print(df)

In [ ]:
ins = strategy.instrument
p = ins.make_price(Decimal(1.12000))
r = ins.make_price(Decimal(1.12055))
q = ins.make_qty(Decimal(1123.123))

In [ ]:
p = ins.make_price(Decimal(1.12000))
r = ins.make_price(Decimal(1.12055))
p.raw
r.raw

pip_risk = 55 #(r-p)
point_value_per_unit = ins.price_increment * ins.lot_size

acc_risk =  Decimal(100)
lots = (acc_risk / pip_risk) * (1 / point_value_per_unit)
qty = lots * ins.lot_size
qty = ins.make_qty(qty)

class RiskCalculator:

    @staticmethod
    def qty_from_risk(risk: Decimal, entry: Decimal, exit: Decimal, ins: Instrument) :
        risk_points = Decimal( abs(entry - exit))
        print(f"risk_points: {risk_points}")
        point_value_per_unit = ins.price_increment * ins.lot_size
        lots = (risk / risk_points) * (1 / point_value_per_unit)
        qty = lots * ins.lot_size
        return ins.make_qty(qty)
    

RiskCalculator.qty_from_risk(Decimal(100), Decimal(1.12000), Decimal(1.12055), ins)